<h1><center><b>Compte-Rendu Intermediaire</b></center></h1>

## Bilan
Je me présente je m'appelle Henri. Je voudrai bien réussir ma vi être aiméééé !

## Formalisation du problème

Soit G = (V, A) un graphe orienté avec : 

*   |V | = n
*   |A| = m
	
Soit R un ensemble de ressources avec |R| = $n_r$.
A chaque arc (i,j) de A, on associe un cout c<sub>i,j</sub>  (on suppose positif ) et un vecteur de consommation de ressource t<sub>i,j</sub>   de dimension $n_r$  
On considère des contraintes de ressources finales portant uniquement sur la borne supérieure.

**Objectif:**
On cherche a trouver un chemin qui minimise le coût et dont la consommation de ressource (noté T) respecte les contraintes.

### Definition

* **Etiquette** : vecteur associé à un chemin et dont les coordonnées sont le coût et les consommations des différentes ressources.
* **Orde de Pareto** : Une étiquette E domine E' ssi toute les cordonnées de E sont supérieur ou égale à celle de E' et qu’au moins une coordonnée de E est strictement supérieur à celle de E'

## Algorithme de résolution du PCCC
On utilisera **l'algorithme à correction d'étiquettes**. 
On considère un graphe de source le sommet s et de puit le sommet t.
A chaque sommet i on associe un ensemble d'étiquettes E<sub>s,i</sub> qui correspondes à des chemins entre la source et ce sommet.
A l'initialisation, tous les ensembles E<sub>s,i</sub> sont vides car aucun chemin n'a été traité exépté E<sub>s,s</sub> qui vaut {0}.


On condisere une liste *L* qui contient l'ensemble des sommet dont au moins une étiquette n'a pas encore été propagé, et qu'on met a jour a chaque itération.
  
A chaque intération, on choisi un sommet de *L* et on propage ses étiquette à ses successeurs si cette propagation respecte les contraintes. On élimine ce sommet de *L*. On ajoute ainsi de nouvelles étiquettes à ses successeur sur lesquelles on effectuera un test de domination au sens de l'ordre de Pareto afin de ne conserver que les étiquettes non-dominé. Ce test de domination permet d'éventuellement **corriger** les étiquettes déjà existantes , d'où le nom de l'algorithme.

A chaque fois qu'on crée une nouvelle étiquette qui est maintenu après le test de domination, le sommet qui lui correspond est ajouté à la liste *L*. L'algorithme s'arrête une fois que la liste est vide donc lorsque tous les sommets sont traités et que les propagations ne change plus les étiquettes.

Sur tous les sommets et particulièrement en t, l'algorithme permet d'obtenir une ou plusieurs étiquettes non dominées  donc candidates comme solutions du problème.
Etant donné que le but est de minimiser le cout de s à t, l'algorithme renvoie le chemin ayant le cout le plus faible en t. S'il y en a plusieurs, on en choisit un. 
Notons que dans cet algorithme, on calcule le plus court chemin contraint de s à tous les autres sommets du graphe.

![Algorithme](https://drive.google.com/uc?id=1GNPnNF0-vO_jykku-rCmQeSDKuthGv9E)














## Modélisation et génération du graph
Nous avons décidé de modéliser nos graphs à l'aide de la POO. Vous pouvez voir ci-dessous la classe *Graph* dont chaque instance contient un attribut *ressource* (liste contenant l'ensemble des ressources) et un attribut *graph_dict* (dictionnaire contenant l'ensemble des arcs avec leurs coût et leur consommation de ressource)


In [0]:
import numpy as np

"""
    Modélisation d'un graphe : classe graphe
    Le graphe sera modélisé sous la forme d'un double-dictionnaire 
    arc[S1 : sommet origine][S2 : sommet but ] = vecteur 
    Si le vecteur existe, alors il y a un arc entre S1 et S2
    Le vecteur sera de taille 1+R et représentera le vecteur cout et ressources de l'arc
    vecteur[0] = cout de l'arc
    vecteur[i] = Consommation ressource i sur l'arc ou i = [| 1 ; R |]
    
"""
class Graphe:
    nb_graph=0

    #res = nom des ressources à considérer
    def __init__(self,res=None):
        self.arc={} #double dictionnaire 
        self.noeud=[] #noeud
        if(type(res) is not list):
          res=[res]
        self.ressource=res # nom des ressources
        #Nombre de ressource
        if(res==[None]):
          self.nb_ressources = 0
        else:  
          self.nb_ressources = len(res) 
        Graphe.nb_graph+=1
    
    #Fonction permettant d'ajouter une/des ressources
    def ajout_ressource(self,nom_ress):
      if (type(nom_ress) is not list):
        nom_ress=[nom_ress]
      for nom in nom_ress:
        if(self.ressource==[None]):
          self.ressource=[nom]
        else: 
          self.ressource.append(nom)
        self.nb_ressources+=1
        for source in self.arc.keys():
          for cible in self.arc[source].keys():
            self.arc[source][cible].append(0)
        
    #Fonction permettant l'ajout d'un/des sommet au graphe
    def ajoutNoeud(self,sommet):
      if (type(sommet) is not list):
        sommet=[sommet]
      for som in sommet:
        if (som in self.noeud):
          print('Le sommet ',som,' est déjà présent dans votre graph')
        else:
          self.noeud.append(som)
    
    #Fonction permettant de connaitre le nombre d'arcs du graphe
    def nb_arcs(self):
      res=0
      for key1 in self.arc:
        for key2 in self.arc[key1].keys():
          res+=1
      return(res)
    
    #Fonction permettant l'ajout d'un/des arc entre 2 sommets
    def ajoutArc(self,origin,dest,conso):
        #conso = Cout de l'emprunt de l'arc et consommation des ressources
        if(type(origin) is not list):
          origin=[origin]
        if(type(dest) is not list):
          dest=[dest]
        if (type(conso) is not list):
          conso=[conso]
        if(type(conso[0]) is not list):
          conso=[conso]
        for i in range(len(origin)):
          if(len(conso[i])==self.nb_ressources+1):
            if(origin[i] not in self.noeud):
                self.ajoutNoeud(origin[i])
            if(origin[i] not in self.arc.keys()):
                self.arc[origin[i]]={}
            if (dest[i] not in self.noeud):
              self.ajoutNoeud(dest[i])
            self.arc[origin[i]][dest[i]]=conso[i]
          else:
            print("Le vecteur conso de l'arc (",origin[i],dest[i],") n'a pas la bonne dimention. L'arc n'a donc pas été ajouté") 
        
    #Fonction retournant les predecesseurs d'un sommet sous forme de liste    
    def predecesseurs(self,sommet):
        liste_pred = []
        for key in self.arc.keys():
            if(sommet in self.arc[key].keys()):
                liste_pred.append(key)  
        return(liste_pred)
        
    #Fonction retournant les successeurs d'un sommet sous forme de liste
    def successeurs(self,sommet):
        liste_suc = []
        for key in self.arc[sommet].keys():
            liste_suc.append(key)
        return(liste_suc)
    
    #Fonction affichant le graphe sous forme
    # Sommet(nombre de successeurs) : [{successeur_i : vecteur conso }, ... ]
    def affiche(self):
        for key in self.arc.keys():
            print(key+"("+str(len(self.arc[key]))+"): [ ",end="")
            for cle in self.arc[key].keys():
                print("{"+cle+" : "+str(self.arc[key][cle])+" } ,",end="")
            print("]")


In [0]:
g=Graphe()
g.ajout_ressource("fatigue")
g.ajout_ressource(["prix essence","cris des enfants"])
g.ajoutNoeud("paris")
g.ajoutNoeud(["Berlin","Tokyo"])
g.ajoutArc("Paris","Berlin",[1,2,3,4])
g.ajoutArc(["Paris","Tokyo"],["Paris","Paris"],[[1,2,3,4],[4,3,2,1]])
print(g.ressource)
print(g.noeud)
print(g.arc)
g.affiche()
g.nb_arcs()

['fatigue', 'prix essence', 'cris des enfants']
['paris', 'Berlin', 'Tokyo', 'Paris']
{'Paris': {'Berlin': [1, 2, 3, 4], 'Paris': [1, 2, 3, 4]}, 'Tokyo': {'Paris': [4, 3, 2, 1]}}
Paris(2): [ {Berlin : [1, 2, 3, 4] } ,{Paris : [1, 2, 3, 4] } ,]
Tokyo(1): [ {Paris : [4, 3, 2, 1] } ,]


3

In [0]:
""" Création d'une fonction Pareto qui prend comme paramètre 2 arcs et 
    renvoie l'arc qui domine au sens de Pareto, ou les 2 arcs si pas 
    de dommination. Un vecteur domine si il minimise la consommation de ressources    """
    
def ordre_Pareto(arc1,arc2):
    arc = arc1 - arc2
    retour = None
    if all(arc>=0)==True:
        retour =[arc2.tolist()]
    elif all(arc<0)==True:
        retour = [arc1.tolist()]
    else:
        retour = [arc1.tolist(),arc2.tolist()]
    return(retour)

def Pareto(E):
   #enleve les valeurs dupliques
   resu=[]
   for i in E:
     if i not in resu:
       resu.append(i)
   #Conserve uniquement les element non dominés
   for element1 in resu:
     for element2 in resu:
       if(element1!=element2):
        test=ordre_Pareto(np.array(element1),np.array(element2))
        if(len(test)==1):
          if element1==test[0]:
            resu.remove(element2)
          else:
            resu.remove(element1)
     return(resu)

def correc_etiquette(g,contraintes):
  if type(contraintes) is not list:
    
  if(g.nb_ressources



In [0]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
arr[np.array([not np.all(arr[i]==[1,2,3,4]) for i in range(3)])]
np.append(arr[1],0)

array([5, 6, 7, 8, 0])

# Résolution par PuLP du modèle de PLNE

In [0]:
from pulp import *
prob = LpProblem("PLNE", LpMinimize)


Ici, on s'intéressera à la modélisation en PLNE du problème de PCCC (Plus Court Chemin avec Contraintes de ressources). Le problème a été formalisé en première partie du notebook. On cherche le plus court chemin de s à t en respectant les contraintes de ressources.
On considère également les contraintes de ressources finales donc pour chaque contrainte, on aura un majorant sur tout le chemin.

**Variables**  
Pour tout $a \in A$ : $x_{a} \in ${$0,1$}$ = 1 $ si et seulement si $a$ est dans le chemin choisi  

**Objectif**  
Minimiser le coût du chemin:  
$ {min} \sum \limits_{{a \in A}} c_a*x_a $

**Contraintes**  
Conservation du flot:  $ \forall i \ne s,t $ on a  $ \sum \limits_{(i,j)\in \delta^+(i)} x_{ij} = \sum \limits_{(j,i)\in \delta^-(i)} x_{ji}   $

Flot entrant de t:  $ \sum \limits_{(i,t) \in \delta^-(t)} x_{it} = 1 $  
Flot sortant de s:  $ \sum \limits_{(s,i) \in \delta^+(s)} x_{si} = 1 $  
Contraintes de ressources sur les arcs:
$ \forall r\in ${$1,2,\dots, n_r $}, $\sum \limits_{a \in A} t^r_a*x_a \le b^r$ où $b^r$ est la consommation maximale de la ressource $r$.






In [0]:
graphe = Graphe()
x = LpVariable("x",(graphe.nb_arcs(),graphe.nb_arcs()),0,1,LpInteger)
